# Fixing Database Error FRQ 2 Extra
> Fixing error caused in transaction due to id generation method in database

- title: Fixing Database Error
- author: Don Tran
- badges: true
- toc: true
- categories: [pbl, sql]

## Error
### Relationships
OneToMany relationship and ManyToOne relationship stores POJOs in separate tables which are tied by one table that stores the relationship between using their respective ids. (In this case I used StepLog instead of Day because I thought it might've been a more appropriate name)

![](https://user-images.githubusercontent.com/75715248/204440921-f752729c-8371-4d0a-95c9-670d4af1a966.png)
![](https://user-images.githubusercontent.com/75715248/204440990-c5a33508-e734-4870-b797-0d634ec0abc6.png)
![](https://user-images.githubusercontent.com/75715248/204441053-12ef2a29-9867-45a9-be62-defb791673ff.png)

### Database Locking
The issue is that SQL's implementation of autogenerated ids comes from hibernate_sequence which is a global table so when a request to pull a value from the table is processed, SQL locks the entire database. So, when a step log is created, JPA requests to generate an id for the newly created steplog which causes the entire database to become locked and in turn prevent the following request to update the person_step_log table from being updated. This results in the transaction failing which rolls back all changes.

![](https://user-images.githubusercontent.com/75715248/204443080-337a2551-af2f-471e-9417-8d53e9fa1298.png)

## Fix
### ID generation using IDENTITY
One fix was to use a separate id generation method independent of the hibernate_sequence table used to automatically generate ids. To do this, you can change the id generation method for StepLog to IDENTITY. 

![](https://user-images.githubusercontent.com/75715248/204443214-00e88e5a-d2bf-4d03-9caf-f45cf7ce2822.png)

### AutoIncreament
This enables you to use SQL's AutoIncreament feature as your id generation method. 
![](https://user-images.githubusercontent.com/75715248/204444678-1c58486c-d2ec-4058-b571-a6d9ce3c7c3d.png)
![](https://user-images.githubusercontent.com/75715248/204444838-d869d854-bb92-401d-ab16-a89f464089e5.png)

### Successful Database Write
Thus, no request is needed/made which locks the database and therefore allows the relationship to be written into the person_step_log table.

![](https://user-images.githubusercontent.com/75715248/204445020-c27e588f-6da0-4e2c-a292-bd3d158488db.png)
![](https://user-images.githubusercontent.com/75715248/204445130-1c5dc268-bb6e-4304-852e-e65355b93546.png)